<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/poly_encoder_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/Poly-Encoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Mon Jul  4 11:01:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install -q transformers datasets folium==0.2.1 apex

In [5]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 84 (delta 12), reused 16 (delta 6), pack-reused 57
Unpacking objects: 100% (84/84), done.


In [6]:
%cd Poly-Encoder

/content/Poly-Encoder


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoModel, AutoTokenizer, AutoConfig, BertPreTrainedModel, BertModel, BertConfig, BertTokenizer

import os
import time
import json
import shutil
import argparse
import numpy as np
from tqdm import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer, AutoConfig, BertPreTrainedModel, BertModel, BertConfig, BertTokenizer, BertTokenizerFast
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

from dataset import SelectionDataset
from transform import SelectionSequentialTransform, SelectionJoinTransform, SelectionConcatTransform
from encoder import PolyEncoder, BiEncoder, CrossEncoder

from sklearn.metrics import label_ranking_average_precision_score

import logging

In [8]:
!gdown --folder 1uC1pSCrh9xlieF60z78QuCg7OxvU9kUs
!mv /content/dstc7/*.json /content/Poly-Encoder/dstc7
!mv /content/dstc7/*.tsv /content/Poly-Encoder/dstc7

Retrieving folder list
Processing file 1YXsj5U-P1nj3ID2ni62vvW4m_4AMYh_v ubuntu_dev_subtask_1.json
Processing file 1I46tBMSYrDCFb0UoequBSl4Uqo0GOVms ubuntu_responses_subtask_1.tsv
Processing file 1tHAe_WGFqQUHmdM28RttlVsYukL69qtg ubuntu_test_subtask_1.json
Processing file 1s2Fz0wQD-YL0tR14PmgdYGuuI7pLarFS ubuntu_train_subtask_1.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1YXsj5U-P1nj3ID2ni62vvW4m_4AMYh_v
To: /content/Poly-Encoder/dstc7/ubuntu_dev_subtask_1.json
100% 92.8M/92.8M [00:00<00:00, 374MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I46tBMSYrDCFb0UoequBSl4Uqo0GOVms
To: /content/Poly-Encoder/dstc7/ubuntu_responses_subtask_1.tsv
100% 101k/101k [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tHAe_WGFqQUHmdM28RttlVsYukL69qtg
To: /content/Poly-Encoder/dstc7/ubuntu_test_subtask_1.json
100% 18.4M/18.4M [00:00<00:00, 293MB/s]
Download

In [9]:
%cd /content/Poly-Encoder/dstc7
!bash parse.sh
%cd /content

/content/Poly-Encoder/dstc7
tcmalloc: large alloc 1859141632 bytes == 0x28ba000 @  0x7f5383ec61e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x5939af 0x594cd3 0x594f8e 0x59526e 0x5bfba0 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f5383ac3c87 0x5b621a
tcmalloc: large alloc 1859141632 bytes == 0x715be000 @  0x7f5383ec61e7 0x4a3940 0x52ab72 0x527cf3 0x51d358 0x59358d 0x548c51 0x51566f 0x549576 0x4bcb19 0x59c019 0x59588e 0x595e64 0x4d8924 0x5bfbcb 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f5383ac3c87 0x5b621a
/content


In [10]:
# model_name = 'klue/bert-base'

In [11]:
# bert_config = BertConfig.from_pretrained(model_name)

In [12]:
# bert = BertModel.from_pretrained(model_name, config=bert_config)
# tokenizer = BertTokenizer.from_pretrained(model_name)

In [13]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [14]:
logging.basicConfig(level=logging.ERROR)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

def eval_running_model(dataloader, test=False):
    model.eval()
    eval_loss, eval_hit_times = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    r10 = r2 = r1 = r5 = 0
    mrr = []
    for step, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if args.architecture == 'cross':
            text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
            with torch.no_grad():
                logits = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))
        else:
            context_token_ids_list_batch, context_input_masks_list_batch, \
            response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch
            with torch.no_grad():
                logits = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                              response_token_ids_list_batch, response_input_masks_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))
        r2_indices = torch.topk(logits, 2)[1] # R 2 @ 100
        r5_indices = torch.topk(logits, 5)[1] # R 5 @ 100
        r10_indices = torch.topk(logits, 10)[1] # R 10 @ 100
        r1 += (logits.argmax(-1) == 0).sum().item()
        r2 += ((r2_indices==0).sum(-1)).sum().item()
        r5 += ((r5_indices==0).sum(-1)).sum().item()
        r10 += ((r10_indices==0).sum(-1)).sum().item()
        # mrr
        logits = logits.data.cpu().numpy()
        for logit in logits:
            y_true = np.zeros(len(logit))
            y_true[0] = 1
            mrr.append(label_ranking_average_precision_score([y_true], [logit]))
        eval_loss += loss.item()
        nb_eval_examples += labels_batch.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = r1 / nb_eval_examples
    if not test:
        result = {
            'train_loss': tr_loss / nb_tr_steps,
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
            'epoch': epoch,
            'global_step': global_step,
        }
    else:
        result = {
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
        }

    return result

In [15]:
args = {
        "bert_model": 'klue/bert-base',
        "eval": False,
        "model_type": 'bert',
        "output_dir": 'output_dstc7/',
        "train_dir": 'dstc7/',

        "use_pretrain": True,
        "architecture": 'poly',

        "max_contexts_length": 128,
        "max_response_length": 32,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "print_freq": 100,

        "poly_m": 16,

        "learning_rate": 5e-5,
        "weight_decay": 0.01,
        "warmup_steps": 100,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1.0,

        "num_train_epochs": 10.0,
        'seed': 12345,
        'gradient_accumulation_steps': 1,
        "fp16": False,
        "fp16_opt_level": "O1",
        'gpu': 0
        }

In [16]:
from easydict import EasyDict as edict
args = edict(args)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "%d" % args.gpu
set_seed(args)

## init dataset and bert model
model_name = 'klue/bert-base'
tokenizer = BertTokenizer.from_pretrained(model_name)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=args.max_contexts_length)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=args.max_response_length)
concat_transform = SelectionConcatTransform(tokenizer=tokenizer, max_len=args.max_response_length+args.max_contexts_length)

print('=' * 80)
print('Train dir:', args.train_dir)
print('Output dir:', args.output_dir)
print('=' * 80)

if not args.eval:
    train_dataset = SelectionDataset(os.path.join(args.train_dir, 'train.txt'),
                                                                    context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'dev.txt'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=1000, mode=args.architecture)
    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size, collate_fn=train_dataset.batchify_join_str, shuffle=True, num_workers=0)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
else: # test
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'test.txt'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)

val_dataloader = DataLoader(val_dataset, batch_size=args.eval_batch_size, collate_fn=val_dataset.batchify_join_str, shuffle=False, num_workers=0)


epoch_start = 1
global_step = 0
best_eval_loss = float('inf')
best_test_loss = float('inf')

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

log_wf = open(os.path.join(args.output_dir, 'log.txt'), 'a', encoding='utf-8')
print(args, file=log_wf)

state_save_path = os.path.join(args.output_dir, '{}_{}_pytorch_model.bin'.format(args.architecture, args.poly_m))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########################################
## build BERT encoder
########################################

bert_config = BertConfig.from_pretrained(model_name)

bert = BertModel.from_pretrained(model_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=args.poly_m)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

if args.eval:
    print('Loading parameters from', state_save_path)
    model.load_state_dict(torch.load(state_save_path))
    test_result = eval_running_model(val_dataloader, test=True)
    print (test_result)
    exit()

no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
)
if args.fp16:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

print_freq = args.print_freq//args.gradient_accumulation_steps
eval_freq = min(len(train_dataloader) // 2, 1000)
eval_freq = eval_freq//args.gradient_accumulation_steps
print('Print freq:', print_freq, "Eval freq:", eval_freq)

for epoch in range(epoch_start, int(args.num_train_epochs) + 1):
    tr_loss = 0
    nb_tr_steps = 0
    with tqdm(total=len(train_dataloader)//args.gradient_accumulation_steps) as bar:
        for step, batch in enumerate(train_dataloader):
            model.train()
            optimizer.zero_grad()
            batch = tuple(t.to(device) for t in batch)
            if args.architecture == 'cross':
                text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
                loss = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch)
            else:
                context_token_ids_list_batch, context_input_masks_list_batch, \
                response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch
                loss = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                        response_token_ids_list_batch, response_input_masks_list_batch,
                                        labels_batch)

            loss = loss / args.gradient_accumulation_steps
            
            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            
            tr_loss += loss.item()

            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                nb_tr_steps += 1
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if nb_tr_steps and nb_tr_steps % print_freq == 0:
                    bar.update(min(print_freq, nb_tr_steps))
                    time.sleep(0.02)
                    print(global_step, tr_loss / nb_tr_steps)
                    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))

                if global_step and global_step % eval_freq == 0:
                    val_result = eval_running_model(val_dataloader)
                    print('Global Step %d VAL res:\n' % global_step, val_result)
                    log_wf.write('Global Step %d VAL res:\n' % global_step)
                    log_wf.write(str(val_result) + '\n')

                    if val_result['eval_loss'] < best_eval_loss:
                        best_eval_loss = val_result['eval_loss']
                        val_result['best_eval_loss'] = best_eval_loss
                        # save model
                        print('[Saving at]', state_save_path)
                        log_wf.write('[Saving at] %s\n' % state_save_path)
                        torch.save(model.state_dict(), state_save_path)
            log_wf.flush()

    # add a eval step after each epoch
    val_result = eval_running_model(val_dataloader)
    print('Epoch %d, Global Step %d VAL res:\n' % (epoch, global_step), val_result)
    log_wf.write('Global Step %d VAL res:\n' % global_step)
    log_wf.write(str(val_result) + '\n')

    if val_result['eval_loss'] < best_eval_loss:
        best_eval_loss = val_result['eval_loss']
        val_result['best_eval_loss'] = best_eval_loss
        # save model
        print('[Saving at]', state_save_path)
        log_wf.write('[Saving at] %s\n' % state_save_path)
        torch.save(model.state_dict(), state_save_path)
    print(global_step, tr_loss / nb_tr_steps)
    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))

Train dir: dstc7/
Output dir: output_dstc7/


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprec

Print freq: 100 Eval freq: 1000


  3%|▎         | 100/3125 [01:47<53:59,  1.07s/it]

100 18.3373264169693


  6%|▋         | 200/3125 [03:35<52:39,  1.08s/it]

200 12.122837131023408


In [ ]:
responses_input_ids = responses_input_ids.view(-1, seq_length)
responses_input_masks = responses_input_masks.view(-1, seq_length)
cand_emb = self.bert(responses_input_ids, responses_input_masks)[0][:,0,:] # [bs, dim]
cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]